<a href="https://colab.research.google.com/github/xborrat/NEFRoHack/blob/main/notebooks/visualize-aki-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
from datetime import timedelta
import os
import sqlite3
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# below imports are used for pretty pandas dataframes and plots
from IPython.display import display, HTML
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 22})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# You only need to authenticate once per session.
auth.authenticate_user()

!pip install sweetviz

In [ ]:
# extract the dataset from bigquery
# Set up environment variables
project_id = 'lcp-internal'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect="standard"
    )

# test it works
df = run_query("""
SELECT *
FROM lcp-internal.aki_study.dataset
""")
assert df['hadm_id'].nunique() == df.shape[0]

for col in ['heartrate_ed', 'sbp_ed', 'dbp_ed']:
  df[col] = pd.to_numeric(df[col], errors='coerce')
display(df.head().T)

In [ ]:
import sweetviz as sv
report = sv.analyze(df, pairwise_analysis='off')

In [ ]:
report.show_notebook()

In [ ]:
# export dataset to CSV and download
df.to_csv('aki_dataset.csv', index=False)
from google.colab import files
files.download('aki_dataset.csv')